In [ ]:
""" Описание столбцов:
Index — номер строки
Suburb — наименование пригорода
Address — адрес
Rooms — количество комнат в помещении
Type — тип здания (h — дом, коттедж, вилла, терраса; u — блочный, дуплексный дом; t — таунхаус)
Price — цена помещения
Method — метод продажи 
SellerG — риэлторская компания
Date — дата продажи (в формате день/месяц/год)
Distance — расстояния до объекта от центра Мельбурна 
Postcode — почтовый индекс
Bedroom — количество спален
Bathroom — количество ванных комнат
Car — количество парковочных мест
Landsize — площадь прилегающей территории
BuildingArea — площадь здания
YearBuilt — год постройки
CouncilArea — региональное управление
Lattitude — географическая широта
Longitude — географическая долгота
Regionname — наименование района Мельбурна
Propertycount — количество объектов недвижимости в районе
Coordinates — широта и долгота, объединённые в кортеж
"""


In [35]:
# базовые операции со столбцами

import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1) # порядковые номера или имена столбцов для удаления, если их несколько, передаётся список
# axis=0 — удаляются строки, axis=1 — удаляются столбцы

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom'] # общее количество комнат
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms # средняя площадь одной комнаты

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area # соотношение площади здания и участка
# <0 площадь участка больше площади здания, -1 без строений, 1 без территорий

In [36]:
# дата и время

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True) # приведение к типу datetime64, dayfirst=True - первым изначально шел день

years_sold = melb_df['Date'].dt.year
# date, year, month, day, time, hour, minute, second, dayofweek, day_name, dayofyear, quarter
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)*100

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') # разница в днях, Series c типом данных timedelta64
delta_days.dt.days # приведение к формату int64

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt'] # возраста объекта недвижимости в годах на момент продажи
melb_df = melb_df.drop('YearBuilt', axis=1) # удаляем дублирующий столбец YearBuilt
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [37]:
# создание и преобразование столбцов

def get_street_type(address): # вытаскиваем подтип улицы из адреса
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type) # применение функции к каждому элементу столбца
popular_stypes = street_types.value_counts().nlargest(10).index # nlargest(n) - возвращает n наибольших значений
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other') 
melb_df = melb_df.drop('Address', axis=1)

pop_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in pop_sellers else 'other')

In [31]:
# задание: опыт работы из hh

import pandas as pd

def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)



experience_col = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
    ])
experience_month = experience_col.apply(get_experience)
print(experience_month)

0    99
1    -1
2    -1
3     3
4    72
dtype: int64


In [48]:
# тип данных Category

unique_list = []
for col in melb_df.columns:
    item = [col, melb_df[col].nunique(),melb_df[col].dtype]
    unique_list.append(item) 
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True) # сортировка DataFrame 

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category') # преобразование типа столбца в Category
melb_df['Regionname'].cat.categories # метод получения информации о значениях
melb_df['Regionname'].cat.codes # каким образом столбец кодируется в виде чисел

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8